In [ ]:
import pandas as pd
import test_groups as TG

Tests = []

In [ ]:
class PrepTests:
    """ Save testname and make a dataframe of test results.
    """
    def __init__(self,name):
        self.name = name
        self.dataframe = self.make_df()

    def make_df(self,add = False, dta =[]):
        if add:
            self.dataframe = pd.concat([self.dataframe, dta], axis= 1, ignore_index= True)
            return
        else:
            df = pd.DataFrame()
            return df

In [8]:
def DataPrep(data):
    for index, element in data.iterrows():
        strings = 0
        if isinstance(element["Sonuç"], str) and element["Sonuç"] != "None":
            strings += 1
            tn = f"Test {element['Test Adı']}"
            tablename = element['Test Adı']
            #tablename= data.iloc[index,0]
            tn =  PrepTests(tablename)
            Tests.append(tn)
        else:
            Tests[-1].make_df(add = True, dta = data.iloc[index])

    for test in Tests:
        test.dataframe = test.dataframe.transpose()

In [ ]:
data = pd.read_excel("Sample.xlsx", header=0)
#data = data.drop(columns= "Referans Değer")
DataPrep(data)

In [ ]:
data.head()

In [ ]:
data.loc[data["Test Adı"] == "Trigliserid", "Referans Değer"]

In [7]:
import re
   
references = "<230 Optimal Düzey >330 Artmış Kardiovasküler Hastalık Riski"

references= references.lower()

string_parts = re.split(r'([<>]=?)', references)
string_parts = [s.strip() for s in string_parts if s.strip()]
givenref = {}
r = re.compile("([0-9]+)([a-zA-Z]+)")

for x in range(0, len(string_parts)):
    if x%2 == 0:
        numeric_pattern = r'[-+]?\d*\.\d+|\d+'
        main = string_parts[x+1]
        numerical_values = re.findall(numeric_pattern, string_parts[x+1])
        numerical_values = [int(num) if num.isnumeric() else float(num) for num in numerical_values]
        main = re.sub(numeric_pattern, '', main).strip() + " " +string_parts[x]
        givenref[main] = numerical_values[0]


givenref
#string_parts

{'optimal düzey <': 230, 'artmış kardiovasküler hastalık riski >': 330}

In [7]:
import re
import test_groups as TG

Group = "ldl_alt_fraksi̇yonları"
test = "ldl1-partikül sayısı"

references = "<140 Optimal Düzey >190 Artmış Kardiovasküler Hastalık Riski"

givenref = {}
#r = re.compile("([0-9]+)([a-zA-Z]+)")

references= references.lower()
string_parts = re.split(r'([<>]=?)', references)
string_parts = [s.strip() for s in string_parts if s.strip()]

for x in range(0, len(string_parts)):
    if x%2 == 0:
        numeric_pattern = r'[-+]?\d*\.\d+|\d+'
        mainstr = string_parts[x+1]
        numerical_values = re.findall(numeric_pattern, mainstr)
        numerical_values = [int(num) if num.isnumeric() else float(num) for num in numerical_values]
        mainstr = re.sub(numeric_pattern, '', mainstr).strip() + " " +string_parts[x]
        givenref[mainstr] = numerical_values[0]

#compare givenref  with the test_group now
refexists = TG.Lead(Group).get(test, None)
if refexists is not None:
    print("This is refexists: ", refexists)
    print("This is givenref: ", givenref)
    try:
        refexists == givenref
    except:
        print(f"Reference of {test} is different than known reference for this test.")
else:
    print(f"No reference found for test: {test}")

This is refexists:  {'optimal düzey <': 140, 'artmış kardiovasküler hastalık riski >': 190}
This is givenref:  {'optimal düzey <': 140, 'artmış kardiovasküler hastalık riski >': 190}


In [9]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import test_groups as TG
import seaborn as sns

cmap_colors = [(0.0000001, 0.3, 0), (0.95, 0.95, 0), (5, 0, 0)]
cmap = mcolors.LinearSegmentedColormap.from_list("custom_color_map", cmap_colors, N=10000)

patient = 150

lwst, bgst, keylwst, keybgst = float('inf'), 0, "", ""
    
for key, value in test.items():
    if value < lwst:
        lwst, keylwst = value, key
    if value > bgst:
        bgst, keybgst = value, key

constant = 2 * (abs(lwst - bgst)) / 3

fig, ax = plt.subplots(figsize=(5, 1))

bars = [(lwst, bgst - lwst), (patient, 0.1)]
ax.broken_barh(bars, (0, 1), facecolors="None")

ax.set_xlim(abs(lwst - constant), bgst + constant)

ax.yaxis.set_ticklabels([])

x_ticks = [lwst, bgst, patient]
x_labels = [f"{keylwst[-1]}{lwst}", f"{keybgst[-1]}{bgst}", str(patient)]

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_labels, rotation="vertical")

ax.get_xticklabels()[0].set_horizontalalignment('right')
ax.get_xticklabels()[1].set_horizontalalignment('left')

ax.imshow(np.arange(0, lwst + bgst + 1).reshape(1, -1), cmap=cmap, aspect='auto', extent=[0, lwst + bgst, 0, 1])

ax.add_patch(plt.Rectangle((0, 0), bgst + constant, 1, fill=False, edgecolor='black', linewidth=2))

plt.grid(False)
plt.show

AttributeError: 'str' object has no attribute 'items'